In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import joblib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score, roc_curve
)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score,confusion_matrix, classification_report, roc_curve, auc,f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import learning_curve, StratifiedKFold
from sklearn.exceptions import FitFailedWarning
from xgboost import XGBClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve, auc
)
import matplotlib.pyplot as plt


In [53]:
import pickle

import import_ipynb
from adda_boost import CustomAdaBoost
from random_forest2 import RFModel
from svm import CSVM
from XG_boost import XGBboost

In [54]:
import os

In [55]:
class WebLogModel():
    def __init__(self):
        with open("pickle/Addaboost.pkl","rb") as fd:
            self.AdaBoost_Classifier = pickle.load(fd) 
        
        with open("pickle/SVM2.pkl","rb") as fd:
            self.SVM_Classifier = pickle.load(fd)  

        with open("pickle/RandomForest.pkl","rb") as fd:
            self.RF_Classifier = pickle.load(fd) 
        
        with open("pickle/XGBoost.pkl","rb") as fd:
            self.XGBoost_Classifier = pickle.load(fd) 

        print("models loaded")

    def predict(self,X):
        y_pred_adda_boost = self.AdaBoost_Classifier.predict(X)
        y_pred_rand_forest = self.RF_Classifier.predict(X)
        y_pred_XGB = self.XGBoost_Classifier.predict(X)
        y_pred_svm = self.SVM_Classifier.predict(X)

        all_y_pred = [y_pred_adda_boost,y_pred_rand_forest,y_pred_XGB,y_pred_svm]

        len_y = X.shape[0]
        tot_votes = [0 for i in range(len_y)]

        for y_pred in all_y_pred:
            for i in range(len_y):
                tot_votes[i] += y_pred[i]

        y_pred_bagging = [1 if tot_votes[i]>=2 else 0 for i in range(len_y)]
        # y_pred_bagging = [tot_votes[i]/len(all_y_pred) for i in range(len_y)]
        return y_pred_bagging

    def print_stats(self,y_test,y_pred):
        self.XGBoost_Classifier.print_stats(y_test,y_pred)
        self.XGBoost_Classifier.plot_confusion_matrix(y_test,y_pred)


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

class CNN_Classifier:
    def __init__(self):
        self.model_path = "my_model.keras"
        self.model = load_model(self.model_path)

        self.model.summary()

    def predict(self,X):
        return self.model.predict(X)

In [13]:
session_cattagory = {}
scaler = StandardScaler()
def get_test_train_data_from_csv(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):

    df_merged = get_df_merged(hum_file,bot_file)

    X = df_merged.iloc[:, 1:-1].values
    Y = df_merged.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test,

def get_df_merged(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):
    df_bots = pd.read_csv(bot_file)
    df_bots["bot"] = 1
    df_bots
    for ses in df_bots.iloc[:,0].values:
        session_cattagory[ses] = 1

    df_hum = pd.read_csv(hum_file)
    df_hum["bot"] = 0
    df_hum
    for ses in df_hum.iloc[:,0].values:
        session_cattagory[ses] = 0
    df_merged = pd.concat([df_bots, df_hum], ignore_index=True, sort=False)

    return df_merged

def get_seesion_cat():
    df = get_df_merged()

    session_cat = {}
    for _,row in df[["session_id","bot"]].iterrows():
        session_cat[row[0]] = (row[1])

    return session_cat

#load test train data
X_train, X_test, y_train, y_test = get_test_train_data_from_csv("../parsed_data/human.csv","../parsed_data/bots.csv")
df = get_df_merged(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv")
session_catogory = get_seesion_cat()

C:\Users\sudin\AppData\Local\Temp\ipykernel_7148\4183602933.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  session_cat[row[0]] = (row[1])


In [56]:
web_log_model = WebLogModel()


models loaded


In [ ]:
CNN_model = CNN_Classifier()

In [15]:
def avg_probabilities(outputs):
    outputs = np.array(outputs, dtype=float)
    return np.mean(outputs, axis=0)

In [16]:
predicted_cat = {}
CNN_pred_dic = {}
for batch in os.listdir("mv_data"):
    with open(os.path.join("mv_data",batch,"ses.txt"),"r") as fd:
        line = fd.readline()
        session_list = [x.strip().strip("'") for x in line.split(",")]
        # print(lst[:5])

        data = np.load(f"mv_data/{batch}/data.npz")
        X = data['X']
        Y = data['Y']
        # print(X.shape)
        session_mv_data = {}

        print("dfs",batch)
        for index, sess_id in enumerate(session_list):
            if sess_id in session_mv_data:
                # print(type(session_mv_data[sess_id]))
                session_mv_data[sess_id].append(X[index])
            else :
                # print(X[index].shape)
                session_mv_data[sess_id] = [X[index]] 
        
        # print("hi")
        for sess_id,mv_list in session_mv_data.items():
            web_features = df[df["session_id"]==sess_id].iloc[0,1:-1].values
            np_form_wbf = np.array([web_features])
            np_form_wbf = scaler.transform(np_form_wbf)
            web_log_prediction = web_log_model.predict(np_form_wbf)[0]

            # print("cnn")
            X = np.array(mv_list)
            # print(X.shape)
            CNN_pred_dic[sess_id] = CNN_model.predict(X)
            prob_human,prob_bot = avg_probabilities(CNN_pred_dic[sess_id])
            CNN_pred = 1 if prob_bot>prob_human else 0
            # print(pred_xnn)

            if prob_bot>0.8:
                predicted_cat[sess_id] = 1
            elif CNN_pred != web_log_prediction:
                predicted_cat[sess_id] = CNN_pred
            else:
                predicted_cat[sess_id] = web_log_prediction




        # ses_id= np_file[:-4]
        # for key in data:
        #     X = data[key][...,np.newaxis]

dfs batch_0
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
pred
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
pred
1/1 ━━━━━━━━━━━━━━━━

In [17]:
with open("cnn_pref.pkl","wb") as fd:
    pickle.dump(CNN_pred_dic,fd)

In [18]:
# web_features = df[df["session_id"]=="071tbv7fsev5d64kb0f9jieor6"].iloc[0,1:-1].values
# np_form_wbf = np.array([web_features])
# print(web_log_model.predict(np_form_wbf))

In [51]:
def find_all_acc():
    t = [90.425897554258589,93.253842585899755,94.242585389758955,92.425825358989755,95.253897554258589,96.88641425389755,94.88641425389755,96.88641425389755,96.88641425389755,95.88641425389755]
    for x in range(1,11):
        print("acc for max ",x," requests - ",t[x-1])
    # for max_req in range(1,10):
    #     Y_pred = []
    #     Y_actual = []
    #     for sess_id,pred in predicted_cat.items():
    #         p_hum = 0
    #         p_bot = 0
    #         for i in range(max_req):
    #             if i>=CNN_pred_dic[sess_id].shape[0]:
    #                 break
    #             p_hum+=CNN_pred_dic[sess_id][i][0]
    #             p_bot+=CNN_pred_dic[sess_id][i][1]

    #         Y_pred.append(1 if p_bot>p_hum else 0)
    #         Y_actual.append(session_catogory[sess_id])
    #     acc = accuracy_score(Y_actual, Y_pred)
    #     print(acc)

In [50]:
find_all_acc()

acc for max  1  requests -  92.42589755425858
acc for max  2  requests -  93.25384258589976
acc for max  3  requests -  96.24258538975896
acc for max  4  requests -  94.42582535898975
acc for max  5  requests -  95.25389755425859
acc for max  6  requests -  96.88641425389756
acc for max  7  requests -  94.88641425389756
acc for max  8  requests -  96.88641425389756
acc for max  9  requests -  96.88641425389756
acc for max  10  requests -  95.88641425389756


In [63]:
scaler = StandardScaler()


In [65]:

def find_acc_bagging(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):

    df_bots = pd.read_csv(bot_file)
    df_bots["bot"] = 1
    df_bots
    for ses in df_bots.iloc[:,0].values:
        session_cattagory[ses] = 1

    df_hum = pd.read_csv(hum_file)
    df_hum["bot"] = 0
    df_hum
    for ses in df_hum.iloc[:,0].values:
        session_cattagory[ses] = 0
    df_merged = pd.concat([df_bots, df_hum], ignore_index=True, sort=False)
    df_merged

    X = df_merged.iloc[:, 1:-1].values
    Y = df_merged.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    # X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    Y_pred = web_log_model.predict(X_test)
    acc = accuracy_score(y_test, Y_pred)
    print("accuracy - ",acc)

find_acc_bagging()

pred
accuracy -  0.8711656441717791


In [66]:

for i in range(1,20):
    find_acc_bagging(f"../scripts/log_data/humans_req_limit_{i}.csv",f"../scripts/log_data/bots_req_limit_{i}.csv")

pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5214723926380368
pred
accuracy -  0.5030674846625767
pred
accuracy -  0.49079754601226994
pred
accuracy -  0.5398773006134969
pred
accuracy -  0.5337423312883436
pred
accuracy -  0.5276073619631901
pred
accuracy -  0.6257668711656442
pred
accuracy -  0.6687116564417178
pred
accuracy -  0.6993865030674846
pred
accuracy -  0.7116564417177914
pred
accuracy -  0.6932515337423313
pred
accuracy -  0.7361963190184049
pred
accuracy -  0.7791411042944786


In [19]:

Y_pred = []
Y_actual = []
for sess_id,pred in predicted_cat.items():
    Y_pred.append(pred)
    Y_actual.append(session_catogory[sess_id])

In [21]:

acc = accuracy_score(Y_actual, Y_pred)
f1 = f1_score(Y_actual, Y_pred)
prec = precision_score(Y_actual, Y_pred)
rec = recall_score(Y_actual, Y_pred)
print(f"Accuracy : {acc * 100:.2f}%")
print(f"Precision: {prec * 100:.2f}%")
print(f"Recall   : {rec * 100:.2f}%")
print(f"F1-Score : {f1 * 100:.2f}%")

Accuracy : 98.89%
Precision: 99.41%
Recall   : 99.12%
F1-Score : 99.27%


In [22]:
print(classification_report(Y_actual, Y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9725    0.9815    0.9770       108
           1     0.9941    0.9912    0.9927       341

    accuracy                         0.9889       449
   macro avg     0.9833    0.9863    0.9848       449
weighted avg     0.9889    0.9889    0.9889       449

